## 1 Import Library

In [1]:
import requests
import time
import pandas as pd

## 2 Parameter Setting

### 2-1  Ref : http://www.patentsview.org/api/query-language.html 

In [2]:
url_post = 'http://www.patentsview.org/api/patents/query'
post_len = 10000

### 2-2 Only Change Parameter : "patent_date" ex) "yyyy-mm-dd"  

In [3]:
#Don't touch ('Only patent_date ex)yyyy-mm-dd')
q1 = '?q={"_and":[ {"_gte":{"patent_date":"2010-01-01"}},{"_or":[{"_text_all":{"patent_title":'
q2 = '}},{"_text_all":{"patent_abstract":'
q3 = '}}]}]}'

In [4]:
#제대로 합쳐졌는지 확인
q = '?q={"_and":[ {"_gte":{"patent_date":"2010-01-01"}},{"_or":[{"_text_all":{"patent_title":search_t}},{"_text_all":{"patent_abstract":search_t}}]}]}'

### 2-3 Patent Keyword Setting

In [5]:
# 2-1) patent keyword
searchText_f = ["autonomous","self-driving","driverless"]
searchText_b = ["car","vehicle","drive","driver","driving"]
searchText_T=[]
for f in searchText_f:
    for b in searchText_b:
        searchText_T.append(f+" "+b)

print(searchText_T)
print(len(searchText_T))

['autonomous car', 'autonomous vehicle', 'autonomous drive', 'autonomous driver', 'autonomous driving', 'self-driving car', 'self-driving vehicle', 'self-driving drive', 'self-driving driver', 'self-driving driving', 'driverless car', 'driverless vehicle', 'driverless drive', 'driverless driver', 'driverless driving']
15


## 3 Patent Request 

### 3-1 Post_Page 는 1page 당 10,000개 뽑혀서 웬만하면 page가 2이상 안넘어갈텐데 넘어가면 밑에 for문이 돌아감.

In [6]:
# post total page list
total_patent = []

# post data
for t,search_t in enumerate(searchText_T):
    print("search num: ",t)
    print("search text: ", search_t)
    st = time.time()
    pt_dict_list = []
    # post first page
    try:
        q=q1+'"'+searchText_T[t]+'"'+q2+'"'+searchText_T[t]+'"'+q3
        f = '&f=["patent_number","patent_title","patent_date","ipc_section","ipc_class","ipc_subclass","patent_type","patent_abstract"]'
        o = '&o={"page":1,"per_page":10000}'
        o1 = '&o={"page":'
        o3 = ',"per_page":10000}'
        o = o1+str(t+1)+o3
        #print(q)
        t_data = q+f+o
        #print(t_data)
        t_post = requests.post(url_post+t_data).json()
        #print(t_post)
        print("total_patent_num :",t_post['total_patent_count'])
        pt_dict_list.extend(t_post['patents'])
        post_page = int(t_post['total_patent_count']/post_len)+2
        print("post page: ",post_page)
        
    except:
        print("=== Try error1 ===")
    
    # post total page
    if post_page >2:                   
        for n in range(2,post_page):
            
            print("page number :",n,"/",post_page)
            try:
                #data = post_data(n,post_len,patent_date,search_t)
                js_data_post = requests.post(url_post+t_data).json()
                pt_dict_list.extend(js_data_post['patents'])
            
            except:
                print("=== Try error2 ===")
            
            
    et = time.time()
    print("exe_time :",et-st)
    time.sleep(1)   
    total_patent.extend(pt_dict_list)

search num:  0
search text:  autonomous car
=== Try error1 ===


NameError: name 'post_page' is not defined

In [72]:
# Check Total Patent
print(len(total_patent))

7322


## 4 Filter Ipc List

In [ ]:
# 1) filter util patent (not design etc..)
util_patent = []
for pat in total_patent:

    if pat['patent_type'] == 'utility':
          util_patent.append(pat)

print(util_patent[0])

In [ ]:
# 2) ipcs ->list
st = time.time()
for d in util_patent:
    ipc_list = []
    for c in d['IPCs']:
        # ==== ipc class prob ['g1c','g5d' ... ] ===== #
        if len(str(c['ipc_class']))==1:
            c['ipc_class'] = str('0') + c['ipc_class']

        ipc = c['ipc_section'] + str(c['ipc_class']) + c['ipc_subclass']
        ipc_list.append(ipc)
    d['ipc_list'] = sorted(ipc_list)

et = time.time()
print("exe_time :",et-st)

print(util_patent[0])

In [ ]:
# dict to dataframes
df = pd.DataFrame.from_dict(util_patent).drop(['IPCs'],axis=1)

print(df.iloc[0])
print('='*50)
print(df['ipc_list'][0])

In [ ]:
df.head()

## 5 Save to CSV File

In [ ]:
df.to_csv("Self_Driving_Patent_recent_1years.csv",encoding='utf8')